In [ ]:
import gradio as gr
import PyPDF2
import docx
import random
from transformers import pipeline

# Load LLM (small one for demo)
chatbot_model = pipeline("text-generation", model="distilgpt2")

# Store sessions for mock interview
interview_sessions = {}

# --- PDF extractor ---
def extract_text_from_pdf(file_path):
    text = ""
    with open(file_path, "rb") as f:
        reader = PyPDF2.PdfReader(f)
        for page in reader.pages:
            page_text = page.extract_text()
            if page_text:
                text += page_text + "\n"
    return text.strip()

# --- DOCX extractor ---
def extract_text_from_docx(file_path):
    doc = docx.Document(file_path)
    text = "\n".join([para.text for para in doc.paragraphs if para.text.strip()])
    return text.strip()

# --- Universal extractor ---
def extract_text(file_path):
    if file_path.endswith(".pdf"):
        return extract_text_from_pdf(file_path)
    elif file_path.endswith(".docx"):
        return extract_text_from_docx(file_path)
    return ""

# --- Generate personalized questions ---
def generate_summary_and_questions(cv_file, jd_file):
    if cv_file is None or jd_file is None:
        return "Please upload both a CV and Job Description.", []

    cv_text = extract_text(cv_file)
    jd_text = extract_text(jd_file)

    prompt = f"""
You are an interview coach.

Candidate CV:
{cv_text}

Job Description:
{jd_text}

1. List the top 5 skills from the CV.
2. List the top 5 key requirements from the JD.
3. Generate 5 personalized interview questions that test the candidate on the JD requirements using the CV context.
Format clearly:
- CV Skills
- JD Requirements
- Interview Questions
    """

    response = chatbot_model(
        prompt,
        max_length=600,
        num_return_sequences=1,
        do_sample=True,
        temperature=0.7
    )[0]['generated_text']

    # Extract questions (simple heuristic: take last 5 numbered lines)
    lines = response.split("\n")
    interview_questions = [line for line in lines if line.strip().startswith(("1", "2", "3", "4", "5"))]

    return response.replace(prompt, "").strip(), interview_questions[-5:]

# --- Feedback generator ---
def give_feedback(answer, question):
    prompt = f"As an interview coach, evaluate this answer.\nQuestion: {question}\nAnswer: {answer}\nGive short feedback with strengths and weaknesses."
    response = chatbot_model(
        prompt,
        max_length=200,
        num_return_sequences=1,
        do_sample=True,
        temperature=0.7
    )[0]['generated_text']
    return response.replace(prompt, "").strip()

# --- Mock interview logic ---
def chat_with_bot(message, history):
    session_id = id(history)
    if session_id not in interview_sessions:
        return "⚠️ Please start the mock interview first."

    session = interview_sessions[session_id]
    q_index = session["current"]
    questions = session["questions"]

    # Evaluate user’s answer
    last_question = questions[q_index]
    feedback = give_feedback(message, last_question)

    # Next question or finish
    session["current"] += 1
    if session["current"] < len(questions):
        next_question = questions[session["current"]]
        return f"💡 Feedback: {feedback}\n\nNext question: {next_question}"
    else:
        del interview_sessions[session_id]
        return f"💡 Feedback: {feedback}\n\n✅ Interview finished. Great job!"

# --- Start mock interview ---
def start_interview(questions):
    if not questions:
        return "⚠️ No questions available. Upload CV + JD first."
    session_id = random.randint(1000, 9999)  # dummy ID
    interview_sessions[session_id] = {"questions": questions, "current": 0}
    return f"🎤 Starting personalized mock interview.\n\nFirst question: {questions[0]}"

# --- UI with Tabs ---
with gr.Blocks() as demo:
    gr.Markdown("## 🤖 CV + JD Powered Mock Interview Coach")

    with gr.Tabs():
        # Tab 1: Upload CV & JD
        with gr.Tab("Upload & Analyze"):
            gr.Markdown("### 📄 Upload CV & Job Description to Generate Questions")
            with gr.Row():
                cv_upload = gr.File(label="Upload CV (PDF or DOCX)", type="filepath")
                jd_upload = gr.File(label="Upload Job Description (PDF or DOCX)", type="filepath")

            output = gr.Textbox(label="Extracted Skills, Requirements & Questions", lines=20)
            role_questions = gr.State([])

            generate_btn = gr.Button("Analyze CV + JD & Generate Questions")
            generate_btn.click(
                fn=generate_summary_and_questions,
                inputs=[cv_upload, jd_upload],
                outputs=[output, role_questions]
            )

        # Tab 2: Mock Interview
        with gr.Tab("Mock Interview"):
            gr.Markdown("### 🎤 Start a Personalized Mock Interview")
            start_btn = gr.Button("Start Mock Interview")
            start_output = gr.Textbox(label="Interview Status")

            start_btn.click(fn=start_interview, inputs=[role_questions], outputs=start_output)

            chatbot = gr.ChatInterface(
                fn=chat_with_bot,
                title="Interview Coach",
                description="Answer the questions. Bot will give feedback and move to the next."
            )

demo.launch()


In [ ]:
!pip install PyPDF2
!pip install python-docx
!pip install transformers